# Restoration and degradation widget
##### The purpose of this notebook is to format the Typology_and_Restoration_Potential table to produce tables that can be used by the API. The data models we need should have the following format:

**Restoration potential:**  
{  
location_id [str]  
indicator [str] - restorable_area, mangrove_area, restoration_potential_score  
value [numeric]  
unit [str] - ha, %  
}  

where:

restorable_area = Tot_Restor  
mangrove_area  = Total_2016  
restoration_potential_score  =  Mean_Score  

**Degradation:**  
{  
location_id [str]  
indicator [str] - degraded_area, lost_area, main_loss_driver  
value [numeric]  
unit [str] - ha, %  
}  

where:

degraded_area = Rest_Area_Dgrd  
lost_area = Area_loss_ha  
main_loss_driver = Loss_Driver


### Steps:
1. Import packages and data
2. Format restoration potential table:
- Get countries for the table
- Give each country their corresponding iso code
- Add staging ids
- Create indicators
- Table cleaning
- Save table
3. Format degradation table:
- Get countries for the table
- Give each country their corresponding iso code
- Add staging ids
- Create indicators
- Table cleaning
- Save table

#### 1. Import packages and data

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import fiona


In [2]:
path_in = '../../../../data/Typology_and_Restoration_Potential/Data'

In [3]:
# Import geodatabase and get Global_Mangrove_Restoration_Jurisdiction_Envelope
gdb_file =f'{path_in}/MOW_Global_Mangrove_Restoration_20190411.gdb'
layers = fiona.listlayers(gdb_file)

In [4]:
layers

['Global_Mangrove_Restoration_Jurisdiction_Envelope',
 'Global_Mangrove_Restoration_Typology']

In [5]:
layer0 = gpd.read_file(gdb_file, driver='FileGDB', layer=0)

In [6]:
layer0.head()

,Country,Tot_Restor,Restor_pct,Rest_Area_Loss,Area_loss_pct,Rest_Area_Dgrd,Area_dgrd_pct,Total_2016,Mean_Score,SOC,AGB,People,Fish_Score,Fish_Score_Inv,Shape_Length,Shape_Area,Loss_Driver,geometry
0,American Samoa,0.00000,0,0.00,0,0.00,0,18.74,0,0.000000,0.000000,0,109000000.0,200000.0,5.350212e+04,1.638363e+07,No Data,"MULTIPOLYGON (((-19011686.866 -1613074.241, -1..."
1,Angola,580.11556,4,671.43,5,68.92,0,13286.05,62,220274.238512,30809.263129,3800,-1.0,-1.0,1.383794e+06,2.785231e+10,NPC,"MULTIPOLYGON (((1550210.491 -1230695.594, 1549..."
2,Anguilla,0.08128,4,1.27,59,0.00,0,0.87,24,407.670000,6.524752,0,1000000.0,2000000.0,2.097797e+04,1.419573e+06,No Data,"MULTIPOLYGON (((-7025249.376 2058315.576, -702..."
3,Antigua and Barbuda,14.15571,2,20.19,2,0.00,0,886.30,22,9031.556968,1078.042000,500,8000000.0,13000000.0,1.888816e+05,1.361710e+09,No Data,"MULTIPOLYGON (((-6873186.951 1921601.093, -688..."
4,Aruba,19.38207,36,20.55,38,0.00,0,33.79,46,9816.581150,1374.874165,0,71000000.0,119000000.0,3.798229e+04,2.166772e+07,No Data,"MULTIPOLYGON (((-7781925.061 1393950.107, -778..."


In [7]:
# Import locations to get staging ids
#locations = pd.read_csv('../../../../data/staging_locations.csv')
#locations = locations[locations['location_type'] == 'country']
#locations = locations[['id', 'name', 'iso']].copy()
#locations.head()

### Add (staging) locations

In [8]:
locations_file = 'https://storage.googleapis.com/mangrove_atlas/boundaries/processed/location_final/locations_v3_not_merged_with_old.gpkg'
locations = gpd.read_file(locations_file)
locations = locations[locations['type'] == 'country']
locations.head()

,name,iso,type,area_m2,wdpaid,globalid,perimeter_m,location_idn,coast_length_m,geometry
82,Qatar,QAT,country,3.880224,NaN,{AF97ABE2-6405-4438-A7ED-1494A43DA379},8.392644,06d2e6f9-bc89-59bf-a0e2-ab804e5db9fd,1345769.96,"MULTIPOLYGON (((50.73769 24.93464, 50.73779 24..."
89,Mayotte,MYT,country,5.611808,NaN,{57E86B5B-7EF0-4754-A8D4-A9DC3212D421},10.086238,0750953f-4af9-549b-aeea-329663249a56,291036.71,"POLYGON ((46.63483 -12.96039, 46.63197 -12.969..."
118,Vietnam,VNM,country,90.156489,NaN,{B2A84FBB-34CD-4A51-9463-B9DB2DB62A10},81.714911,09a1ab14-11ad-56ec-8acb-a149e5697abd,9005760.08,"MULTIPOLYGON (((104.31952 10.36051, 104.31975 ..."
132,Grenada,GRD,country,2.154728,NaN,{F8753179-5FFA-4D9E-8AD9-083F31C48528},6.743601,0b0ecb56-bb8e-5ef1-b8ee-3cdad67fed0e,260664.47,"MULTIPOLYGON (((-61.91525 11.37330, -61.91813 ..."
149,India,IND,country,473.029671,NaN,{A4A6CE4D-8D03-4246-9A2F-BD9811232115},211.564078,0c07ca53-7b17-5650-a2c6-0cc27249a4bd,16917891.22,"MULTIPOLYGON (((79.52922 9.38411, 79.52921 9.3..."


In [9]:
api_locs = pd.read_csv('https://storage.googleapis.com/mangrove_atlas/widget_data/locations_staging.csv')
api_locs.rename(columns={'location_id': 'location_idn'}, inplace=True)
api_locs.head()

,id,location_idn
0,1563,000bd204-c0fd-510b-a1ad-132a7ef7470d
1,1564,00250a0f-f66d-54a0-b7a3-d80035881cbf
2,1565,0041637b-f6a2-5b89-87ce-850f5c5431b3
3,1566,005b49ef-6b7f-575a-85b3-ff19261a0755
4,1567,00921349-70fb-5a7e-8207-b3157aecc349


In [10]:
# Import gadm file to retrieve ISO codes
gadm = gpd.read_file('../../../../data/gadm36_level0_original/gadm36_level0_original.shp')
gadm.head()

,GID_0,NAME_0,AREA_KM2,MOL_ID,Shape_Leng,Shape_Area,geometry
0,ABW,Aruba,1.819384e+02,1,0.963634,0.015131,"POLYGON ((-69.97820 12.46986, -69.97847 12.469..."
1,AFG,Afghanistan,6.438575e+05,2,57.103371,62.749594,"POLYGON ((68.52644 31.75435, 68.53852 31.75457..."
2,AGO,Angola,1.247422e+06,3,73.796528,103.818655,"MULTIPOLYGON (((11.73347 -16.67255, 11.73347 -..."
3,AIA,Anguilla,8.330331e+01,4,1.318321,0.007116,"MULTIPOLYGON (((-63.42375 18.58903, -63.42375 ..."
4,ALA,Åland,1.506261e+03,5,42.232199,0.243769,"MULTIPOLYGON (((21.32195 59.74986, 21.32195 59..."


#### 2. Create Restoration Potential table
Get countries for the table


In [11]:
rp = layer0[['Country']]
rp.head()

,Country
0,American Samoa
1,Angola
2,Anguilla
3,Antigua and Barbuda
4,Aruba


Give each country their corresponding iso code


In [12]:
isocodes = gadm[['GID_0', 'NAME_0']]

In [13]:
# Add iso codes to countries in rp dataframe
rp = pd.merge(rp, isocodes, how='left', left_on='Country', right_on='NAME_0')
rp.head()

,Country,GID_0,NAME_0
0,American Samoa,ASM,American Samoa
1,Angola,AGO,Angola
2,Anguilla,AIA,Anguilla
3,Antigua and Barbuda,ATG,Antigua and Barbuda
4,Aruba,ABW,Aruba


In [14]:
# Check which countries did not match with gadm and have no iso
rp[rp['GID_0'].isnull()]

,Country,GID_0,NAME_0
12,Bonaire,NaN,NaN
30,East Timor,NaN,NaN
83,Sao Tome and Principe,NaN,NaN


In [15]:
# Add isocodes to those countries
rp.GID_0[12] = 'BES'
rp.GID_0[30] = 'TLS' # East Timor is now called Timor-Leste and the iso is TLS
rp.GID_0[83] = 'STP'

In [16]:
rp = rp.drop(columns='NAME_0').rename(columns={'GID_0':'iso'})
rp.head()

,Country,iso
0,American Samoa,ASM
1,Angola,AGO
2,Anguilla,AIA
3,Antigua and Barbuda,ATG
4,Aruba,ABW


Add staging ids

In [17]:
# Add codes from locations
rp_locs = pd.merge(rp, locations[['iso', 'location_idn']], on='iso', how='left')
rp_locs = pd.merge(rp_locs, api_locs, on='location_idn', how='left')
#mang_df_final = mang_df_final[mang_df_final.id.notna()]
rp_locs.drop(columns=['location_idn'], inplace=True)
rp_locs

,Country,iso,id
0,American Samoa,ASM,2346.0
1,Angola,AGO,2029.0
2,Anguilla,AIA,1915.0
3,Antigua and Barbuda,ATG,3095.0
4,Aruba,ABW,2707.0
...,...,...,...
103,Vanuatu,VUT,3248.0
104,Venezuela,VEN,3582.0
105,Vietnam,VNM,1681.0
106,"Virgin Islands, U.S.",VIR,2339.0


In [18]:
# Countries with no ID
rp_locs[rp_locs['id'].isnull()]

,Country,iso,id
49,Hong Kong,HKG,NaN


Create indicators

In [19]:
# Add restorable area indicator
ind1 = rp_locs.copy()
ind1['indicator'] = 'restorable_area'
ind1 = pd.merge(ind1,layer0[['Country','Tot_Restor']],on='Country', how='left').rename(columns={'Tot_Restor':'value'})
ind1['unit']='ha'
ind1.head()

,Country,iso,id,indicator,value,unit
0,American Samoa,ASM,2346.0,restorable_area,0.00000,ha
1,Angola,AGO,2029.0,restorable_area,580.11556,ha
2,Anguilla,AIA,1915.0,restorable_area,0.08128,ha
3,Antigua and Barbuda,ATG,3095.0,restorable_area,14.15571,ha
4,Aruba,ABW,2707.0,restorable_area,19.38207,ha


In [20]:
# Add mangrove area indicator
ind2 = rp_locs.copy()
ind2['indicator'] = 'mangrove_area'
ind2 = pd.merge(ind2,layer0[['Country','Total_2016']],on='Country', how='left').rename(columns={'Total_2016':'value'})
ind2['unit']='ha'
ind2.head()

,Country,iso,id,indicator,value,unit
0,American Samoa,ASM,2346.0,mangrove_area,18.74,ha
1,Angola,AGO,2029.0,mangrove_area,13286.05,ha
2,Anguilla,AIA,1915.0,mangrove_area,0.87,ha
3,Antigua and Barbuda,ATG,3095.0,mangrove_area,886.30,ha
4,Aruba,ABW,2707.0,mangrove_area,33.79,ha


In [21]:
# Add restoration potential score indicator
ind3 = rp_locs.copy()
ind3['indicator'] = 'restoration_potential_score'
ind3 = pd.merge(ind3,layer0[['Country','Mean_Score']],on='Country', how='left').rename(columns={'Mean_Score':'value'})
ind3['unit']=''
ind3.head()

,Country,iso,id,indicator,value,unit
0,American Samoa,ASM,2346.0,restoration_potential_score,0,
1,Angola,AGO,2029.0,restoration_potential_score,62,
2,Anguilla,AIA,1915.0,restoration_potential_score,24,
3,Antigua and Barbuda,ATG,3095.0,restoration_potential_score,22,
4,Aruba,ABW,2707.0,restoration_potential_score,46,


In [22]:
# Merge all indicators in same table
restoration = pd.concat([ind1, ind2, ind3], ignore_index=True)
restoration


,Country,iso,id,indicator,value,unit
0,American Samoa,ASM,2346.0,restorable_area,0.00000,ha
1,Angola,AGO,2029.0,restorable_area,580.11556,ha
2,Anguilla,AIA,1915.0,restorable_area,0.08128,ha
3,Antigua and Barbuda,ATG,3095.0,restorable_area,14.15571,ha
4,Aruba,ABW,2707.0,restorable_area,19.38207,ha
...,...,...,...,...,...,...
319,Vanuatu,VUT,3248.0,restoration_potential_score,64.00000,
320,Venezuela,VEN,3582.0,restoration_potential_score,57.00000,
321,Vietnam,VNM,1681.0,restoration_potential_score,54.00000,
322,"Virgin Islands, U.S.",VIR,2339.0,restoration_potential_score,29.00000,


Table cleaning

In [23]:
# Countries with no staging id
restoration[restoration.id.isna()]

,Country,iso,id,indicator,value,unit
49,Hong Kong,HKG,NaN,restorable_area,15.093,ha
157,Hong Kong,HKG,NaN,mangrove_area,462.200,ha
265,Hong Kong,HKG,NaN,restoration_potential_score,36.000,


In [24]:
restoration_final = restoration [['id', 'indicator', 'value', 'unit']].copy()
restoration_final.unit = 'ha'
restoration_final.rename(columns={'id':'location_id'}, inplace=True)


Save table

In [25]:
restoration_final[~restoration_final.location_id.isna()].to_csv('../../../data/RestorationPotential.csv', index = False)

#### Format degradation table


Create indicators

In [25]:
# Create degraded area indicator
ind1 = rp_locs.copy()
ind1['indicator'] = 'degraded_area'
ind1 = pd.merge(ind1,layer0[['Country','Rest_Area_Dgrd']],on='Country', how='left').rename(columns={'Rest_Area_Dgrd':'value'})
ind1['unit']='ha'
ind1.head()

,Country,iso,id,indicator,value,unit
0,American Samoa,ASM,2346.0,degraded_area,0.00,ha
1,Angola,AGO,2029.0,degraded_area,68.92,ha
2,Anguilla,AIA,1915.0,degraded_area,0.00,ha
3,Antigua and Barbuda,ATG,3095.0,degraded_area,0.00,ha
4,Aruba,ABW,2707.0,degraded_area,0.00,ha


In [26]:
# Create lost area indicator
ind2 = rp_locs.copy()
ind2['indicator'] = 'lost_area'
ind2 = pd.merge(ind2,layer0[['Country','Rest_Area_Loss']],on='Country', how='left').rename(columns={'Rest_Area_Loss':'value'})
ind2['unit']='ha'
ind2.head()

,Country,iso,id,indicator,value,unit
0,American Samoa,ASM,2346.0,lost_area,0.00,ha
1,Angola,AGO,2029.0,lost_area,671.43,ha
2,Anguilla,AIA,1915.0,lost_area,1.27,ha
3,Antigua and Barbuda,ATG,3095.0,lost_area,20.19,ha
4,Aruba,ABW,2707.0,lost_area,20.55,ha


In [27]:
# Add mangrove area indicator
ind3 = rp_locs.copy()
ind3['indicator'] = 'mangrove_area'
ind3 = pd.merge(ind3,layer0[['Country','Total_2016']],on='Country', how='left').rename(columns={'Total_2016':'value'})
ind3['unit']='ha'
ind3.head()

,Country,iso,id,indicator,value,unit
0,American Samoa,ASM,2346.0,mangrove_area,18.74,ha
1,Angola,AGO,2029.0,mangrove_area,13286.05,ha
2,Anguilla,AIA,1915.0,mangrove_area,0.87,ha
3,Antigua and Barbuda,ATG,3095.0,mangrove_area,886.30,ha
4,Aruba,ABW,2707.0,mangrove_area,33.79,ha


In [28]:
# Add mangrove area indicator
ind4 = rp_locs.copy()
ind4['indicator'] = 'restorable_area'
ind4 = pd.merge(ind4,layer0[['Country','Tot_Restor']],on='Country', how='left').rename(columns={'Tot_Restor':'value'})
ind4['unit']='ha'
ind4.head()

,Country,iso,id,indicator,value,unit
0,American Samoa,ASM,2346.0,restorable_area,0.00000,ha
1,Angola,AGO,2029.0,restorable_area,580.11556,ha
2,Anguilla,AIA,1915.0,restorable_area,0.08128,ha
3,Antigua and Barbuda,ATG,3095.0,restorable_area,14.15571,ha
4,Aruba,ABW,2707.0,restorable_area,19.38207,ha


In [29]:
# Create main loss driver indicator
ind5 = rp_locs.copy()
ind5['indicator'] = 'main_loss_driver'
ind5 = pd.merge(ind5,layer0[['Country','Loss_Driver']],on='Country', how='left').rename(columns={'Loss_Driver':'main_loss_driver'})
ind5= ind5.replace({'main_loss_driver':{'No Data':'undefined'}})
ind5 = ind5[['iso', 'main_loss_driver']]
ind5.head()

,iso,main_loss_driver
0,ASM,undefined
1,AGO,NPC
2,AIA,undefined
3,ATG,undefined
4,ABW,undefined


In [30]:
# Merge all indicators in same table
degradation = pd.concat([ind1, ind2, ind3,ind4], ignore_index=True)
degradation_driver = pd.merge(degradation, ind5, on='iso', how = 'left')
degradation_driver

,Country,iso,id,indicator,value,unit,main_loss_driver
0,American Samoa,ASM,2346.0,degraded_area,0.00000,ha,undefined
1,Angola,AGO,2029.0,degraded_area,68.92000,ha,NPC
2,Anguilla,AIA,1915.0,degraded_area,0.00000,ha,undefined
3,Antigua and Barbuda,ATG,3095.0,degraded_area,0.00000,ha,undefined
4,Aruba,ABW,2707.0,degraded_area,0.00000,ha,undefined
...,...,...,...,...,...,...,...
427,Vanuatu,VUT,3248.0,restorable_area,9.91022,ha,undefined
428,Venezuela,VEN,3582.0,restorable_area,12019.52468,ha,Erosion
429,Vietnam,VNM,1681.0,restorable_area,17405.43423,ha,Commodities
430,"Virgin Islands, U.S.",VIR,2339.0,restorable_area,3.76000,ha,undefined


Table cleaning


In [31]:
degradation_final = degradation_driver[['id', 'indicator', 'value', 'unit', 'main_loss_driver']].copy()
degradation_final.value = np.int64(degradation_final.value)
degradation_final.rename(columns={'id':'location_id'}, inplace=True)
degradation_final.unit= 'ha'


Save table

In [133]:
degradation_final[~degradation_final.location_id.isna()].to_csv('../../../../data/Degradation_UPDATED.csv', index = False)